# Task 1


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
import hashlib
import random

In [ ]:
def preprocess_document(document):
    preprocess_document = document
    return preprocess_document

# UDF for preprocessing
preprocess_udf = udf(preprocess_document, StringType())

In [ ]:
class InMemoryMinHashLSH:
    def __init__(self, documents):
        self.documents = documents

    @staticmethod
    def preprocess_document(document):
        return document.strip()

    def shingling(self):
        shingle_size = 5
        self.documents['preprocessed_document'] = self.documents['document'].apply(self.preprocess_document)

        def generate_shingles(document):
            shingles = set()
            words = document.split()
            for i in range(len(words) - shingle_size + 1):
                shingle = " ".join(words[i:i + shingle_size])
                shingles.add(shingle)
            return list(shingles)

        self.documents['shingles'] = self.documents['preprocessed_document'].apply(generate_shingles)

    def minhashing(self):
        num_perm = 128

        def minhash_vector(shingles):
            signature = [float('inf')] * num_perm
            for shingle in shingles:
                for j in range(num_perm):
                    hash_value = hash(shingle) ^ hash(j)
                    signature[j] = min(signature[j], hash_value)
            return signature

        self.documents['signature'] = self.documents['shingles'].apply(minhash_vector)

    def locality_sensitive_hashing(self):
        signatures = self.documents['signature'].tolist()
        num_buckets = 1000
        buckets = {}

        for signature in signatures:
            bucket_id = hash(tuple(signature)) % num_buckets
            if bucket_id not in buckets:
                buckets[bucket_id] = []
            buckets[bucket_id].append(signature)

        self.hashed_buckets = [(bucket_id, buckets[bucket_id]) for bucket_id in buckets]

    def run(self):
        self.shingling()
        self.minhashing()
        self.locality_sensitive_hashing()

    def approxNearestNeighbors(self, query_document, n):
        query_shingles = set(self.preprocess_document(query_document).split())
        nearest_neighbors = []

        for bucket_id, bucket_signatures in self.hashed_buckets:
            for signature in bucket_signatures:
                similarity = self.jaccard_similarity(set(signature), query_shingles)
                nearest_neighbors.append((signature, similarity))

        nearest_neighbors.sort(key=lambda x: x[1], reverse=True)
        return nearest_neighbors[:n]

    def jaccard_similarity(self, set1, set2):
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))

        return intersection / union

In [ ]:
with open('/content/drive/MyDrive/Colab_Notebooks/WebOfScience-5736.txt', 'r') as file:
    lines = file.readlines()

In [ ]:
documents = [{'document': line.strip()} for line in lines]
df = pd.DataFrame(documents)

In [ ]:
minhash_lsh = InMemoryMinHashLSH(df)

# Run the MinHashLSH algorithm
minhash_lsh.run()

In [ ]:
query_document = "Background: Leukotoxin (Ltx) expressed by Aggregatibacter actinomycetemcomitans is a powerful exotoxin, which can cause imbalance in host response. Immunoreactivity to Ltx is a marker for presence of leukotoxic A. actinomycetemcomitans, a presence that may modify the disease pattern of colonized individuals. The aim of the present study is to examine presence of systemic immunoreactivity to A. actionmycetemcomitans Ltx with respect to clinical and inflammatory findings in individuals with or without periodontitis (N = 88). Methods: Periodontal status was examined in patients with severe"
nearest_neighbors = minhash_lsh.approxNearestNeighbors(query_document, n=10)

In [ ]:
# Calculate Jaccard similarity between the given document and each document in the dataset
similarities = []
for index, row in df.iterrows():
    set1 = set(minhash_lsh.preprocess_document(query_document).split())
    set2 = set(minhash_lsh.preprocess_document(row['document']).split())
    similarity = minhash_lsh.jaccard_similarity(set1, set2)
    similarities.append((index, similarity))

# Sort the similarities in descending order
similarities.sort(key=lambda x: x[1], reverse=True)

# Print the top 10 most similar documents
print("Top 10 most similar documents:")
for i, (index, similarity) in enumerate(similarities[:10], 1):
    print(f"{i}. Document {index + 1}: Similarity = {similarity}")

Top 10 most similar documents:
1. Document 342: Similarity = 0.5981308411214953
2. Document 341: Similarity = 0.152317880794702
3. Document 5735: Similarity = 0.14393939393939395
4. Document 5043: Similarity = 0.14285714285714285
5. Document 753: Similarity = 0.1417910447761194
6. Document 5746: Similarity = 0.1417910447761194
7. Document 1324: Similarity = 0.14166666666666666
8. Document 6317: Similarity = 0.14166666666666666
9. Document 969: Similarity = 0.13846153846153847
10. Document 5962: Similarity = 0.13846153846153847


## Task 2

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import explode

In [ ]:
class LargeScaleMinHashLSH:
    def __init__(self, spark):
        self.spark = spark
        self.documents = None
        self.shingles = None
        self.signatures = None
        self.hash_buckets = None

    def shingling(self, documents):
        self.shingles = documents.select("doc_id", "text") \
            .rdd.map(lambda x: (x[0], x[1].split())) \
            .toDF(["doc_id", "shingles"])

    def minhashing(self, documents, num_hash_functions=100):
        exploded_shingles = self.shingles.select("doc_id", explode("shingles").alias("shingle"))
        hash_values = exploded_shingles.select("shingle") \
            .distinct().rdd.map(lambda x: (x[0], [hash(x[0]) % num_hash_functions for _ in range(num_hash_functions)])) \
            .toDF(["shingle", "hash_values"])


        self.signatures = exploded_shingles.join(hash_values, exploded_shingles.shingle == hash_values.shingle) \
            .groupBy("doc_id").agg(collect_list("hash_values").alias("hash_values"))

    def locality_sensity_hashing(self, documents, num_hash_buckets=10):
        self.hash_buckets = self.signatures.rdd.flatMap(lambda x: [(tuple(h), x[0]) for h in x[1]]) \
            .map(lambda x: ((hash(x[0]), x[1]), x[0])) \
            .groupByKey().map(lambda x: (x[0][0] % num_hash_buckets, [x[0][1]])) \
            .reduceByKey(lambda x, y: x + y).collect()

    def run(self):
        self.shingling(self.documents)
        self.minhashing(self.documents)
        self.locality_sensity_hashing(self.documents)

    def approxNearestNeighbors(self, documents, query_document, n):
        query_hash = hash(query_document)
        for bucket_id, documents in self.hash_buckets:
            if hash(query_hash) % len(self.hash_buckets) == bucket_id:
                return documents[:n]

        return []

    def jaccard_similarity(self, set1, set2):
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))

        return intersection / union



In [ ]:
spark = SparkSession.builder \
    .appName("MinHashLSH_WebOfScience") \
    .getOrCreate()

In [ ]:
def read_file(path):
  with open(path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    return spark.createDataFrame([(i, line.strip()) for i, line in enumerate(lines)], ["doc_id", "text"])

file_content_df = read_file('/content/drive/MyDrive/Colab_Notebooks/WebOfScience-5736.txt')
file_content_df.head()

Row(doc_id=0, text='Phytoplasmas are insect-vectored bacteria that cause disease in a wide range of plant species. The increasing availability of molecular DNA analyses, expertise and additional methods in recent years has led to a proliferation of discoveries of phytoplasma-plant host associations and in the numbers of taxonomic groupings for phytoplasmas. The widespread use of common names based on the diseases with which they are associated, as well as separate phenetic and taxonomic systems for classifying phytoplasmas based on variation at the 16S rRNA-encoding gene, complicates interpretation of the literature. We explore this issue and related trends through a focus on Australian pathosystems, providing the first comprehensive compilation of information for this continent, covering the phytoplasmas, host plants, vectors and diseases. Of the 33 16Sr groups reported internationally, only groups I, II, III, X, XI and XII have been recorded in Australia and this highlights the need 

In [ ]:
lsh = LargeScaleMinHashLSH(spark)
lsh.documents = file_content_df

lsh.run()


query_document = "Background: Leukotoxin (Ltx) expressed by Aggregatibacter actinomycetemcomitans is a powerful exotoxin, which can cause imbalance in host response. Immunoreactivity to Ltx is a marker for presence of leukotoxic A. actinomycetemcomitans, a presence that may modify the disease pattern of colonized individuals. The aim of the present study is to examine presence of systemic immunoreactivity to A. actionmycetemcomitans Ltx with respect to clinical and inflammatory findings in individuals with or without periodontitis (N = 88). Methods: Periodontal status was examined in patients with severe "

n = 10
results = lsh.approxNearestNeighbors(file_content_df, query_document, n)
print("Approximate nearest neighbors:", results)

Approximate nearest neighbors: [1453, 1485, 3407, 3873, 4681, 5306, 26, 63, 63, 100]


In [ ]:
query_shingles = set(query_document.split())

for result_doc_id in results:

    # print(file_content_df.filter(file_content_df['doc_id'] == result_doc_id).collect()[0]['text'])
    result_shingle_set = set(file_content_df.filter(file_content_df['doc_id'] == result_doc_id).collect()[0]['text'])
    jaccard_similarity = lsh.jaccard_similarity(result_shingle_set, query_shingles)
    # print(result_shingle_set.intersection(query_shingles))
    print(jaccard_similarity)

0.009009009009009009
0.009615384615384616
0.008849557522123894
0.01680672268907563
0.017241379310344827
0.015267175572519083
0.009433962264150943
0.018867924528301886
0.018867924528301886
0.008695652173913044


In [ ]:
# Stop SparkSession
spark.stop()

In [18]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/ProgressII.ipynb"

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/ProgressII.ipynb to html
[NbConvertApp] Writing 627063 bytes to /content/drive/MyDrive/Colab Notebooks/ProgressII.html
